In [1]:
%reset-f
%load_ext autoreload
%autoreload
%matplotlib inline

In [2]:
from pandas.io import wb
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
%autoreload
from res_ind_lib import *
import os, time
import warnings
warnings.filterwarnings("always",category=UserWarning)

# Data

###  from the world bank

In [3]:
nb_weeks = (time.time()-os.stat("wb_data.csv").st_mtime )/(3600*24*7)
if nb_weeks>10: 
    warnings.warn("World bank data are "+str(int(nb_weeks))+" weeks old. You may want to download them again.")
df=pd.read_csv("wb_data.csv").set_index("country")
df.dropna().shape

(73, 12)

In [4]:
#Income gorups for countries and HIPC status
groups = pd.read_csv("income_groups.csv",header =4,index_col=2).replace({"ROM":"ROU","ZAR":"COD"})  #they used old iso3 country for Zair (Congo) and Romania
groups= groups[["Code","Region","Income group","Lending category","Other"]]
groups.dropna(axis=0,how="all",inplace=True);
groups.rename(columns={"Other":"HIPC"},inplace=True)
groups['HIPC'].fillna('norm',inplace=True)
groups["Lending category"].replace("..","",inplace=True)

df["iso3"]=groups["Code"]


In [5]:
def names_to_wb_name(df_in,any_name_to_iso3,iso3_to_unique_name,common_name="iso3",end_name="country"):
    """Matches arbitrary country names from a dataset to world bank country names thru iso3 codes"""
    
    df=pd.DataFrame(df_in)
    
    index_name=df.index.name
    
    df[common_name]=any_name_to_iso3[common_name]
    
    #warns and prints if some countries are not recognized
    cond = df[common_name].isnull()
    if cond.sum()>0:
        warnings.warn("Unrecognized countries in 1st argument 'df':"+", ".join((df.index[cond].values)))
        warnings.warn("Correct 'df' or add lines to 2nd argument 'any_name_to_iso3'")
    
    df=df.dropna()
    
    #warning if missing
    df =df.reset_index().set_index(common_name)
    df[end_name]=iso3_to_unique_name
    cond = df[end_name].isnull()
    if cond.sum()>0:
        warnings.warn("this countries appear to be missing from 3rd argument 'iso3_to_unique_name':"+", ".join((df.index[cond])))

           
    #return df with original indexing
    return df.dropna().set_index(index_name)

### Definitions

In [6]:
# share of poor individuals, we chose the bottom 20
ph=df["pov_head"]=0.2 

### HFA

gets hfa data

In [7]:
# READ THE LAST HFA DATA
hfa_newest =   pd.read_csv("HFA_all_2011_2013.csv").rename(columns={"ISO 3":"iso3","Country name":"country"}).set_index("country")

# READ THE PREVIOUS HFA DATA
hfa_previous = pd.read_csv("HFA_all_2009_2011.csv").rename(columns={"ISO 3":"iso3","Country name":"country"}).set_index("country")

#most recent values... if no 2011-2013 reporting, we use 2009-2011
hfa=hfa_newest.fillna(hfa_previous)

# access to early warning = priority for action 2, indicator 3 of the HFA
hfa["shew"]=1/5*hfa["P2-C3"] # between zero and 1. 

# ability to scale up = average of priority for action 5 of the HFA
#hfa["prepare_scaleup_old"]=(hfa["P5-C1"]+hfa["P5-C2"]+hfa["P5-C4"])/3/5 # between zero and 1
hfa["prepare_scaleup"]=(hfa["P4-C2"]+hfa["P5-C2"]+hfa["P4-C5"])/3/5 # between zero and 1
hfa["finance_pre"] = hfa["P5-C3"]/5 #betwenn 0 and 1


matches names with wb names

In [8]:
#this table matches country names rendered in several ways to their actual iso2 and iso3 codes
any_name_to_iso3 =pd.read_csv("names_to_iso.csv").set_index("country")
#this tables has WB country names and iso3 countries
iso3_to_unique_name = pd.read_csv("iso3_to_wb_name.csv").set_index("iso3")

#matches names in HFA to  names
df[["shew","prepare_scaleup","finance_pre"]]= names_to_wb_name(hfa[["shew","prepare_scaleup","finance_pre"]],any_name_to_iso3,iso3_to_unique_name) 

# Assumes that no HFA reporting means no preparation
df[["shew","prepare_scaleup","finance_pre"]] = df[["shew","prepare_scaleup","finance_pre"]].fillna(0)

### EUsilc and other countries

In [9]:
silc = pd.read_csv("social_ratios.csv")
#EU cuntry code to iso2 
silc["iso2"]=silc["cc"].replace({"EL":"GR","UK":"GB"}) #Greece and UK
silc.set_index("iso2",inplace=True)

#iso3 to wb country name table
iso3_to_wb=pd.read_csv("iso3_to_wb_name.csv").set_index("iso3")

#iso2 to iso3 table
iso2_iso3 = pd.read_csv("names_to_iso.csv")[["iso2","iso3"]].drop_duplicates().set_index("iso3") #the tables has more lines than countries to account for several ways of writing country names

#iso2 to WB 
iso2_iso3["country"]=iso3_to_wb["country"]
iso2_country=iso2_iso3.reset_index().set_index("iso2")

#indexes this data by WB country
silc["country"]=iso2_country["country"]
silc.set_index("country",inplace=True)
#silc.drop_duplicates(inplace=True)

#Monitoring differences between SILC and ASPIRE
hop=df.ix[silc.index,["social_p","social_r"]].dropna().rename(columns=dict(social_p = "s_p_aspire", social_r="s_r_aspire"))
hop[["sp_silc","sr_silc"]]= silc.ix[hop.index,["social_p","social_r"]]
#plt.scatter(x=hop.sp_silc,y=hop.s_p_aspire), plt.scatter(x=hop.sr_silc,y=hop.s_r_aspire,c="red")

df.ix[silc.index,["social_p","social_r"]]=silc[["social_p","social_r"]]

### Country Ratings

In [10]:
#Reads the data and check its not too old
the_credit_rating_file ="cred_rat.csv"
nb_weeks = (time.time()-os.stat(the_credit_rating_file).st_mtime )/(3600*24*7)
if nb_weeks>3: 
    warnings.warn("Credit ratings are "+str(int(nb_weeks))+" weeks old. Get new ones at http://www.tradingeconomics.com/country-list/rating")
ratings_raw  =pd.read_csv(the_credit_rating_file,dtype="str").dropna(how="all")
ratings_raw=ratings_raw.rename(columns={"Unnamed: 0": "country_in_ratings"}).set_index("country_in_ratings")[["S&P","Moody's","Fitch"]]
ratings_raw.rename(index=str.strip,inplace=True)

#this table matches country names rendered in several ways to their actual iso2 and iso3 codes
names_to_iso =pd.read_csv("names_to_iso.csv").set_index("country")

#this tables has WB country names and iso3 countries
iso_country = pd.read_csv("iso3_to_wb_name.csv").set_index("iso3")

ratings_raw =  names_to_wb_name(ratings_raw,names_to_iso,iso_country)
#ratings_raw=ratings_raw.reset_index().set_index("country")

def mystriper(string):
    """strip blanks and converts everythng to lower case"""
    if type(string)==str:
        return str.strip(string).lower()
    else:
        return string
        
#to lower case and strips blanks
ratings_raw=ratings_raw.applymap(mystriper)    

#Transforms ratings letters into 1-100 numbers
rat_disc = pd.read_csv("cred_rat_dict.csv")
ratings=ratings_raw
ratings["S&P"].replace(rat_disc["s&p"].values,rat_disc["s&p_score"].values,inplace=True)
ratings["Moody's"].replace(rat_disc["moodys"].values,rat_disc["moodys_score"].values,inplace=True)
ratings["Fitch"].replace(rat_disc["fitch"].values,rat_disc["fitch_score"].values,inplace=True)
df["rating"]=ratings.mean(axis=1)/100

df["rating"].fillna(0,inplace=True)  #assumes no rating is bad rating


C:\Users\Admin\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:5: UserWarning: Credit ratings are 4 weeks old. Get new ones at http://www.tradingeconomics.com/country-list/rating


### Capital data

In [11]:
#data arranged from Penn tables
k_data=pd.read_csv("capital_data.csv")[["code","cgdpo","ck"]].replace({"ROM":"ROU","ZAR":"COD"}).rename(columns={"cgdpo":"prod_from_k","ck":"k"})#Zair is congo

#matches names in the dataset with world bank country names
iso_country = pd.read_csv("iso3_to_wb_name.csv").set_index("iso3")
k_data.set_index("code",inplace=True)
k_data["country"]=iso_country["country"]
cond = k_data["country"].isnull()
if cond.sum()>0:
     warnings.warn("this countries appear to be missing from iso3_to_wb_name.csv: "+" , ".join(k_data.index[cond].values))
k_data=k_data.reset_index().set_index("country")

# average productivity of capital
df["avg_prod_k"]=k_data["prod_from_k"]/k_data["k"]


###Hazard (protection)

In [12]:
#assumed a function of the income group
protection_assumptions = pd.read_csv("protection_level_assumptions.csv").set_index("Income group")
df["protection"]=groups["Income group"].replace(protection_assumptions["protection"].to_dict())

### Exposure (population in flood-prone areas)

In [13]:
#Esposure data 
expo = pd.read_csv("people_affected_flood.csv").set_index("ISO")[["people_affected_RP10flood_percent"]]

iso3_to_wb_name = pd.read_csv("iso3_to_wb_name.csv").set_index("iso3")
expo["country"]=iso3_to_wb_name["country"]

df["faref"]=df["fa"]=expo.reset_index().dropna().set_index("country")[["people_affected_RP10flood_percent"]].dropna()

#Exposure bias
pe=df["peref"] =df["pe"]=.2


### Vulnerability

In [14]:
#Reads data from pager
pager=pd.read_csv("pager_vulnerabilities.csv")[["ISO-3digit","R_1","R_2","R_3","VV_1","VV_2","VV_3","Country Name"]].set_index("ISO-3digit")

#indexes pager data by country        
pager["country"]=iso3_to_wb_name["country"]
cond = pager["country"].isnull()
if cond.sum()>0:
     warnings.warn("this countries appear to be missing from iso3_to_wb_name.csv: "+" , ".join(pager.index[cond].values)+".\nThese are: "+" , ".join(pager.ix[cond,"Country Name"].values) )
pager["country"]=iso3_to_wb_name["country"]
hop=pager.dropna().reset_index().set_index("country")

#sorts vulnerabilites according to income
share =hop[["R_1","R_2","R_3"]]
damrat=hop[["VV_1","VV_2","VV_3"]]

damrat.columns=["R_1","R_2","R_3"]
p=(share.cumsum(axis=1)-.20)
p[p<0]=0
poor=share-p
poor[poor<0]=0

vp=(poor*damrat).sum(axis=1)/poor.sum(axis=1)
rich=share-poor

vr=(rich*damrat).sum(axis=1)/rich.sum(axis=1)


C:\Users\Admin\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:8: UserWarning: this countries appear to be missing from iso3_to_wb_name.csv: ALA , ATA , BES , BVT , IOT , CXR , CCK , FLK , GUF , ATF , GIB , GLP , GGY , HMD , VAT , JEY , MTQ , MYT , MSR , NFK , PCN , REU , BLM , SHN , SPM , SGS , SJM , TKL , UMI , WLF , ESH.
These are: Aland Islands , Antarctica , Bonaire , Bouvet Island , British Indian Ocean Territory , Christmas Island , the Cocos (Keeling) Islands , the Falkland Islands (Malvinas) , French Guiana , the French Southern Territories , Gibraltar , Guadeloupe , Guernsey , Heard Island and McDonald Islands , the Holy See (Vatican City State) , Jersey , Martinique , Mayotte , Montserrat , Norfolk Island , Pitcairn , Reunion , Saint Barthelemy , Saint Helena , Saint Pierre and Miquelon , South Georgia and the South Sandwich Islands , Svalbard and Jan Mayen , Tokelau , the United States Minor Outlying Islands , Wallis and Futuna , Western Sahara


In [15]:
#Expresses vulnerability as total and bias

fa=df["fa"]
pe=df["peref"] =df["pe"]
fap=fa*(1+pe)
far=(fa-ph*fap)/(1-ph)

df["share1_ref"]=df["share1"]
cp=   df["share1"] /.2*df["gdp_pc_pp"]
cr=(1-df["share1"])/.8*df["gdp_pc_pp"]

df["v"]  = (ph*vp*cp*fap + (1-ph)*vr*cr*far)/(ph*cp*fap + (1-ph)*cr*far)
df["pv"] =  vp/df.v-1

#vulnerability of diversified (shared) capital
df["v_s"]=vr



### Homogenity of losses

In [16]:
#resds loss distribution in mumbai
hop=pd.read_csv("losses_distribution_mumbai.csv",index_col=0)["losses/asset_cost"]
# standard deviation of the underlying normal distribution
s = np.log(hop).std() 

df["H"]= np.exp (-s**2/2)


## Report missing data for the national studies

In [17]:
def write_missing_data(s):
    which = s[s.isnull()].index.values
    return ", ".join(which)

def count_missing_data(s):
    return s.isnull().sum()

report = pd.DataFrame()

report["nb_missing"]=df.apply(count_missing_data,axis=1)  
report["missing_data"]=df.apply(write_missing_data,axis=1)

report  = report.ix[report["nb_missing"]>0,:]
report.sort(columns="nb_missing",inplace=True)
report.to_csv("missing_data_report.csv")

report.head()

,nb_missing,missing_data
country,,
Nicaragua,1,avg_prod_k
Djibouti,1,unemp
Togo,1,social_p
Iraq,1,plgp
Afghanistan,2,"plgp, avg_prod_k"


## manual parametrization

In [18]:
# no ripple effects
df["alpha"] =0 

#Reconstruction time
df["T_rebuild_K"] = 3
df["T_rebuild_L"] = 40

# how much early warning reduces vulnerability
df["pi"] = 0.2

#income elasticity
df["income_elast"] = 1.5

##sensitivity anlysis

In [19]:
df["original_country"]=df.index;

country_sens_list=["Malawi","Sweden"]
#reads paramters to be varied
alt_params=pd.read_csv("alt_params.csv")

df_sensit = pd.DataFrame(columns=df.columns)

for c in country_sens_list:
    for p in alt_params.index.tolist():
        param = alt_params.ix[p,"param"]
        new_value = alt_params.ix[p,"value"]
        d = c+" $("+alt_params.ix[p,"formated_name"]+"="+str(new_value)+")$"
        df_sensit.ix[d] = df.ix[c]
        df_sensit.ix[d,param] =new_value
        df_sensit.ix[d,"original_country"]=c
        
'''for c in df.index:
    for param in ["income_elast"]:
        for new_value in [1, 2]:
            d = c+"_"+param+"_"+str(new_value)
            df_sensit.ix[d] = df.ix[c]
            df_sensit.ix[d,param] =new_value
            df_sensit.ix[d,"original_country"]=c '''

df_sensit.index.name="country"; #after adding indices pandas forgets their name

# Computes and saves national resilience

Central cases

In [20]:
#saves orginal dataframe before adding columns with results
df.drop("iso3", axis=1).to_csv("df_original.csv")

In [21]:
df_with_results=compute_resiliences(df)
print("nb countries with all data :"+str(df.dropna().shape[0]))
print("nb countries with all data and results :"+str(df_with_results["resilience"].dropna().shape[0]))
df_with_results.to_csv("df.csv")

nb countries with all data :90
nb countries with all data and results :90


Sensititvity analysis

In [22]:
df_sensit.drop("iso3", axis=1).to_csv("df_sensit_input.csv")

In [23]:
compute_resiliences(df_sensit).to_csv("df_sensit_results.csv")

Basic reporting

In [24]:
#Some stats
a=df_with_results.resilience;
a.min()*100,a.mean()*100,a.max()*100

(24.669693906030894, 57.97149126008749, 81.279007359104355)

In [25]:
#Outputs table with main results
to_output = ['gdp_pc_pp', 'pop',"protection","fa","v","resilience","risk"]
a=df_with_results[to_output]
a.loc[:,["fa","v","resilience","risk"]]=100*a.loc[:,["fa","v","resilience","risk"]]
desc=pd.read_csv("inputs_info.csv").set_index('key')["descriptor"]
a.rename(columns=desc.to_dict()).dropna().to_excel("results.xlsx")


C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\indexing.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Admin\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#Ex post studies

In [26]:
#Reads the data
expost = pd.read_csv("expost_studies.csv").set_index("place")

#fills the altnerative lines with default values
expost_filled = expost.fillna( value=expost.ix["Mumbai"],axis=0)

#Computes resilience
ep=compute_resiliences(expost_filled,kind="ex-post")

#Report
ep[expost.columns]=expost
ep.to_csv("expost_studies_with_results.csv")


# Simple OLS

In [27]:
from statsmodels.formula.api import ols

In [28]:
explicative_set = ['avg_prod_k', 'axfin_p', 'axfin_r', 'axhealth', 'fa', 'finance_pre', 'gdp_pc_pp', 
                   'pe', 'plgp', 'prepare_scaleup', 'protection', 'pv', 'rating', 'share1',
                   'shew', 'social_p', 'social_r', 'unemp', 'v']

formula ="resilience ~ "+" + ".join(explicative_set) 
olsmodel=ols(formula,data=df_with_results).fit()

a=olsmodel.summary()
f = open('regression_against_intpus.csv', 'w')
f.write(a.as_csv())
f.close()
a

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             resilience   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     42.03
Date:                Tue, 12 May 2015   Prob (F-statistic):           1.10e-30
Time:                        18:45:22   Log-Likelihood:                 163.11
No. Observations:                  90   AIC:                            -288.2
Df Residuals:                      71   BIC:                            -240.7
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           0.4198      0.056      7.542      0.000         0.309     0.531
avg_prod_k         -0.4131      0.046     -8.989      0.000        -0.505    -0.321
axfin_p            -0.0479      0.103     -0.463      0.645        -0.254     0.158
axfin_r             0.0756      0.082      0.922      0.360        -0.088     0.239
axhealth           -0.0751      0.054     -1.383      0.171        -0.183     0.033
fa                  0.1102      0.115      0.959      0.341        -0.119     0.339
finance_pre         0.0115      0.050      0.230      0.819        -0.088     0.111
gdp_pc_pp       -1.173e-06   1.33e-06     -0.880      0.382     -3.83e-06  1.49e-06
pe                  0.0840      0.011      7.542      0.000         0.062     0.106
plgp                0.1091      0.044      2.463      0.016         0.021     0.197
prepare_scaleup     0.0696      0.049      1.419      0.160        -0.028     0.167
protection          0.0001      0.000      0.362      0.718        -0.001     0.001
pv                 -0.0482      0.008     -6.260      0.000        -0.064    -0.033
rating              0.0358      0.031      1.145      0.256        -0.027     0.098
share1              2.2705      0.345      6.588      0.000         1.583     2.958
shew               -0.0184      0.042     -0.441      0.661        -0.102     0.065
social_p            0.1889      0.036      5.231      0.000         0.117     0.261
social_r            0.1141      0.059      1.942      0.056        -0.003     0.231
unemp               0.0594      0.100      0.595      0.554        -0.140     0.258
v                  -0.0007      0.043     -0.016      0.988        -0.087     0.086
==============================================================================
Omnibus:                       15.143   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.824
Skew:                           0.833   Prob(JB):                     8.17e-05
Kurtosis:                       4.498   Cond. No.                     2.29e+22
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.35e-35. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""